# Simple Backup

This is a simple example on how  to use backup. We will run Weaviate Embedded, and will need to enable the [backup-filesystem](https://weaviate.io/developers/weaviate/configuration/backups#filesystem) module at the environment variable `ENABLE_MODULES`  and provide a `BACKUP_FILESYSTEM_PATH` where backups can be created to and restored from.

**Note**: the backup-filesystem is only suitable for single node clusters. For multi-node cluster you need to use a remote bucket, such as Minio, S3, GCP, et c

In [ ]:
import os
import weaviate

client = weaviate.connect_to_embedded(
    version="1.29.0",
    environment_variables={
        "ENABLE_MODULES": "backup-filesystem",
        "BACKUP_FILESYSTEM_PATH": "/tmp/weaviate-backup",
    },
)

In [2]:
# now, let's create our backup folder
os.makedirs("/tmp/weaviate-backup", exist_ok=True)

In [3]:
# those are the collections we have
client.collections.list_all()

{}

In [4]:
# lets create some collections
client.collections.delete(["Collection1", "Collection2"])
c1 = client.collections.create("Collection1")
c2 = client.collections.create("Collection2")
# and insert some content
c1.data.insert({"name": "John Doe"}, vector=[1,2,3])
c2.data.insert({"name": "Mary Jane"}, vector=[3,2,1])

{"build_git_commit":"35d800d9b9","build_go_version":"go1.24.0","build_image_tag":"HEAD","build_wv_version":"1.29.0","level":"warning","msg":"prop len tracker file /Users/dudanogueira/.local/share/weaviate/collection1/3C7soUcZW0ow/proplengths does not exist, creating new tracker","time":"2025-02-19T11:31:00-03:00"}
{"action":"hnsw_prefill_cache_async","build_git_commit":"35d800d9b9","build_go_version":"go1.24.0","build_image_tag":"HEAD","build_wv_version":"1.29.0","level":"info","msg":"not waiting for vector cache prefill, running in background","time":"2025-02-19T11:31:00-03:00","wait_for_cache_prefill":false}
{"build_git_commit":"35d800d9b9","build_go_version":"go1.24.0","build_image_tag":"HEAD","build_wv_version":"1.29.0","level":"info","msg":"Created shard collection1_3C7soUcZW0ow in 1.599125ms","time":"2025-02-19T11:31:00-03:00"}
{"action":"hnsw_vector_cache_prefill","build_git_commit":"35d800d9b9","build_go_version":"go1.24.0","build_image_tag":"HEAD","build_wv_version":"1.29.0","

UUID('cc0c13fe-ef43-4607-8136-f84b9b21085b')

In [5]:
# as we are using backup-filesystem, we need to provide both the backup id and the backend
my_backup_id = "my-new-backup"
backup = client.backup.create(
    backup_id=my_backup_id, 
    backend="filesystem",
    wait_for_completion=False # we don't want to wait for the backup to finish
)
if backup.error:
    print(f"Oh No! Something went wrong while creating {backup.backup_id}")
else:
    print(f"Backup {backup.backup_id} was triggered successfully!")

Backup my-new-backup was triggered successfully!


{"action":"try_backup","backend":"filesystem","backup_id":"my-new-backup","build_git_commit":"35d800d9b9","build_go_version":"go1.24.0","build_image_tag":"HEAD","build_wv_version":"1.29.0","level":"info","msg":"","time":"2025-02-19T11:31:34-03:00","took":3721250}


{"build_git_commit":"35d800d9b9","build_go_version":"go1.24.0","build_image_tag":"HEAD","build_wv_version":"1.29.0","class":"Collection1","level":"info","msg":"start uploading files","time":"2025-02-19T11:31:34-03:00"}
{"build_git_commit":"35d800d9b9","build_go_version":"go1.24.0","build_image_tag":"HEAD","build_wv_version":"1.29.0","class":"Collection1","level":"info","msg":"finish uploading files","time":"2025-02-19T11:31:34-03:00"}
{"backup_id":"my-new-backup","build_git_commit":"35d800d9b9","build_go_version":"go1.24.0","build_image_tag":"HEAD","build_wv_version":"1.29.0","class":"Collection1","level":"info","msg":"release backup","time":"2025-02-19T11:31:34-03:00"}
{"build_git_commit":"35d800d9b9","build_go_version":"go1.24.0","build_image_tag":"HEAD","build_wv_version":"1.29.0","class":"Collection2","level":"info","msg":"start uploading files","time":"2025-02-19T11:31:34-03:00"}
{"build_git_commit":"35d800d9b9","build_go_version":"go1.24.0","build_image_tag":"HEAD","build_wv_vers

In [6]:
# now we can check the status of our backup
backup_status = client.backup.get_create_status(
    backup_id=my_backup_id,
    backend="filesystem"
)

{"action":"backup_status","backend":"filesystem","backup_id":"my-new-backup","build_git_commit":"35d800d9b9","build_go_version":"go1.24.0","build_image_tag":"HEAD","build_wv_version":"1.29.0","level":"info","msg":"","time":"2025-02-19T11:31:47-03:00","took":163541}


In [7]:
print(backup_status)

error=None status=<BackupStatus.SUCCESS: 'SUCCESS'> path='/tmp/weaviate-backup/my-new-backup' backup_id='my-new-backup'


In [8]:
# We can also list
os.listdir(f"/tmp/weaviate-backup/{my_backup_id}")

['backup_config.json', 'Embedded_at_8079']

In [9]:
# now lets read the content of /tmp/weaviate-backup/{my_backup_id}/backup_config.json
# and check the content
import json
with open("/tmp/weaviate-backup/my-new-backup/backup_config.json", "r") as f:
    backup_config = json.load(f)
    print(backup_config)

{'startedAt': '2025-02-19T14:31:34.80344Z', 'completedAt': '2025-02-19T14:31:36.809093Z', 'id': 'my-new-backup', 'nodes': {'Embedded_at_8079': {'classes': ['Collection1', 'Collection2'], 'status': 'SUCCESS', 'error': ''}}, 'node_mapping': None, 'status': 'SUCCESS', 'version': '2.1', 'serverVersion': '1.29.0', 'leader': 'Embedded_at_8079', 'error': ''}


In [10]:
# We can now delete our collections, and restore from that backup
client.collections.delete(["Collection1", "Collection2"])

In [11]:
client.collections.exists("Collection1")

False

In [12]:
# we can specify the collections we want to restore
restore = client.backup.restore(backup_id=my_backup_id, backend="filesystem", include_collections=["Collection1"])

{"action":"try_restore","backend":"filesystem","backup_id":"my-new-backup","build_git_commit":"35d800d9b9","build_go_version":"go1.24.0","build_image_tag":"HEAD","build_wv_version":"1.29.0","level":"info","msg":"","time":"2025-02-19T11:32:20-03:00","took":3130958}
{"action":"restore","backup_id":"my-new-backup","build_git_commit":"35d800d9b9","build_go_version":"go1.24.0","build_image_tag":"HEAD","build_wv_version":"1.29.0","class":"Collection1","level":"info","msg":"successfully restored","time":"2025-02-19T11:32:20-03:00"}
{"action":"restore","backup_id":"my-new-backup","build_git_commit":"35d800d9b9","build_go_version":"go1.24.0","build_image_tag":"HEAD","build_wv_version":"1.29.0","level":"info","msg":"backup restored successfully","time":"2025-02-19T11:32:20-03:00"}


{"action":"restore","backup_id":"my-new-backup","build_git_commit":"35d800d9b9","build_go_version":"go1.24.0","build_image_tag":"HEAD","build_wv_version":"1.29.0","level":"info","msg":"coordinator: backup restored successfully","time":"2025-02-19T11:32:22-03:00"}


In [13]:
client.collections.exists("Collection1")

True

{"action":"hnsw_prefill_cache_async","build_git_commit":"35d800d9b9","build_go_version":"go1.24.0","build_image_tag":"HEAD","build_wv_version":"1.29.0","level":"info","msg":"not waiting for vector cache prefill, running in background","time":"2025-02-19T11:32:23-03:00","wait_for_cache_prefill":false}
{"build_git_commit":"35d800d9b9","build_go_version":"go1.24.0","build_image_tag":"HEAD","build_wv_version":"1.29.0","level":"info","msg":"Completed loading shard collection1_3C7soUcZW0ow in 1.222708ms","time":"2025-02-19T11:32:23-03:00"}
{"action":"hnsw_vector_cache_prefill","build_git_commit":"35d800d9b9","build_go_version":"go1.24.0","build_image_tag":"HEAD","build_wv_version":"1.29.0","count":3000,"index_id":"main","level":"info","limit":1000000000000,"msg":"prefilled vector cache","time":"2025-02-19T11:32:23-03:00","took":40041}


In [14]:
col1 = client.collections.get("Collection1")
print(col1.query.fetch_objects().objects[0].properties)

{'name': 'John Doe'}
